In [1]:
app_dir = {
    "battery":"/home/jczeng/Dropbox/ASE'17/dataset/android/battery_doctor/all_label/",
    "booking":"/home/jczeng/Dropbox/ASE'17/dataset/android/booking_hotel_reservations/all_label/",
#     "cleanmaster":"/home/jczeng/Dropbox/ASE'17/dataset/android/clean_master/all_label/",
    "ebay":"/home/jczeng/Dropbox/ASE'17/dataset/android/ebay/all_label/",
#     "evernote":"/home/jczeng/Dropbox/ASE'17/dataset/android/evernote/all_label/",
    "line":"/home/jczeng/Dropbox/ASE'17/dataset/android/line/all_label/",
    "spotify":"/home/jczeng/Dropbox/ASE'17/dataset/android/spotify_music/all_label/",
#     "swiftkey":"/home/jczeng/Dropbox/ASE'17/dataset/android/swiftkey_keyboard/all_label/",
    "viber":"/home/jczeng/Dropbox/ASE'17/dataset/android/viber/all_label/",
}
veri_file = {
    "cleanmaster": "/home/jczeng/Dropbox/ASE'17/dataset/android/clean_master/changelog_cleanmaster.txt",
    "evernote": "/home/jczeng/Dropbox/ASE'17/dataset/android/evernote/changelog_evernote.txt",
    "swiftkey": "/home/jczeng/Dropbox/ASE'17/dataset/android/swiftkey_keyboard/changelog_swiftkey.txt",
    "battery": "/home/jczeng/Dropbox/ASE'17/dataset/android/battery_doctor/changelog_battery.txt",
    "booking": "/home/jczeng/Dropbox/ASE'17/dataset/android/booking_hotel_reservations/changelog_booking.txt",
    "ebay": "/home/jczeng/Dropbox/ASE'17/dataset/android/ebay/changelog_ebay.txt",
    "line": "/home/jczeng/Dropbox/ASE'17/dataset/android/line/changelog_line.txt",
    "viber": "/home/jczeng/Dropbox/ASE'17/dataset/android/viber/changelog_viber.txt",
    "spotify": "/home/jczeng/Dropbox/ASE'17/dataset/android/spotify_music/changelog_spotify.txt",
}
from os import listdir
from os import path
from nltk.stem.wordnet import WordNetLemmatizer
special_words = ['prosses', 'acsess']
skip_words = ['feed']

def lemmatize(word):
    if word in skip_words:
        return word
    n_word = WordNetLemmatizer().lemmatize(word, 'v')
    if n_word not in special_words:
        temp_token = WordNetLemmatizer().lemmatize(n_word, 'n')
        if (n_word != temp_token and not re.search(r'ss$', n_word)):
            n_word = temp_token
    return n_word

In [4]:
from gensim.models import Word2Vec
import numpy as np
def validation(logfile, label_phrases, emerge_phrases):
    # read changelog
    clog = []
    with open(logfile) as fin:
        for line in fin.readlines():
            line = line.strip()
            issue_kw = map(lambda s: s.strip().split(), line.split(","))
            clog.append(issue_kw)
    # check alignment
    if len(clog) != len(label_phrases):
        logging.error("length not corrected: %d, %d in %s"%(len(clog), len(label_phrases), logfile))
        exit(0)
    # compare topic label using keyword
    # load word2vec model
    wv_model = Word2Vec.load(os.path.join("..", "model", "appreviews_word2vec.model"))
    label_phrase_precisions = []; label_phrase_recalls = []; label_sent_precisions = []; label_sent_recalls = []
    em_phrase_precisions = []; em_phrase_recalls = []; em_sent_precisions = []; em_sent_recalls = []
    # two list: [['keyword1', 'keyword2', ...], ['keyword1', 'keyword2', ...]]
    #           [['label1', 'label2', ...], ['label1', 'label2', ...]]
    for id, ver in enumerate(clog):
        if ver == [[]]: # skip the empty version changelog
            continue
        label_phrase_match_set = set(); label_phrase_issue_match_set = set(); label_sent_match_set = set(); label_sent_issue_match_set = set()
        em_phrase_match_set = set(); em_phrase_issue_match_set = set(); em_sent_match_set = set(); em_sent_issue_match_set = set()

        # phrase
        for issue in ver:
            for kw in issue:
                kw_match = False
                for ws in label_phrases[id]:
                    for w in ws:
                        label_match = False
                        for w_s in w.split("_"):
                            if sim_w(kw, w_s, wv_model) > 0.6:
                                # hit
                                #logging.info("hit: %s -> %s"%(w, kw))
                                label_match = True
                                kw_match = True
                                break
                        if label_match: # if label match found, add label to match set
                            label_phrase_match_set.add(w)
                if kw_match:    # if kw match found, add issue to match set
                    label_phrase_issue_match_set.add("_".join(issue))

        # check emerging issue label
        # merge current version and next version
        if id == 0:     # skip the first epoch
            continue
        if id != len(clog) - 1:
            m_ver = ver + clog[id+1]
        else:
            m_ver = ver
        for issue in m_ver:
            for kw in issue:
                kw_match = False
                for tws in emerge_phrases[id-1]:
                    for w in tws:
                        label_match = False
                        for w_s in w.split("_"):
                            if sim_w(kw, w_s, wv_model) > 0.6:
                                # hit
                                #logging.info("hit: %s -> %s" % (w, kw))
                                label_match = True
                                kw_match = True
                                break
                        if label_match:
                            em_phrase_match_set.add("_".join(tws))
                            break
                if kw_match:
                    em_phrase_issue_match_set.add("_".join(issue))

        # compute
        label_phrase_precision = len(label_phrase_match_set) / float(len(label_phrases[id]))
        label_phrase_recall = len(label_phrase_issue_match_set) / float(len(ver))
        if len(emerge_phrases[id-1]) != 0:
            em_phrase_precision = len(em_phrase_match_set) / float(len(emerge_phrases[id-1]))
            em_phrase_precisions.append(em_phrase_precision)
        em_phrase_recall = len(em_phrase_issue_match_set) / float(len(ver))

        label_phrase_precisions.append(label_phrase_precision)
        label_phrase_recalls.append(label_phrase_recall)
        em_phrase_recalls.append(em_phrase_recall)
    label_phrase_fscore = 2 * np.mean(label_phrase_recalls) * np.mean(em_phrase_precisions) / (np.mean(label_phrase_recalls) + np.mean(em_phrase_precisions))
    label_sent_fscore = 2 * np.mean(label_sent_recalls) * np.mean(em_sent_precisions) / (np.mean(label_sent_recalls) + np.mean(em_sent_precisions))
    print("Phrase label precision: %s\trecall: %f"%(np.mean(label_phrase_precisions), np.mean(label_phrase_recalls)))
    print(
        "Emerging phrase precision: %s\trecall: %f" % (np.mean(em_phrase_precisions), np.mean(em_phrase_recalls)))
    print("Phrase F1 score: %f"%label_phrase_fscore)
    with open("../result/ldaresult/dtm.txt", "a") as fout:
        fout.write("%s\t%f\t%f\t%f\n"%(logfile, np.mean(label_phrase_recalls), np.mean(em_phrase_precisions), label_phrase_fscore))
        
def sim_w(w1, w2, wv_model):
    if w1 not in wv_model or w2 not in wv_model:
        return 0.0
    return wv_model.similarity(w1, w2)       
        

In [5]:
import re
import logging
import os
for apk, filepath in app_dir.items():
    label_files = listdir(filepath)
    label_phrases = []
    print("processing %s"%(apk))
    for ver_file in sorted(label_files, key=lambda s: map(int, s.split('.'))):
        with open(os.path.join(filepath, ver_file)) as fin:
            lines = fin.readlines()
        label_phrase = []
        
        for line in lines:
            line = line.strip()
            line = line.split()
            line = map(lemmatize, line)
            tp_issue = []
            if re.search(r'^[-+]?[0-9]*\.?[0-9]+$', line[-1]):
                phrase = "_".join(line[0:-1])
            else:
                phrase = "_".join(line)
            tp_issue.append(phrase)
            label_phrase.append(tp_issue)
        label_phrases.append(label_phrase)
    # validate
    validation(veri_file[apk], label_phrases, label_phrases)
    print(label_phrases)

processing spotify


/home/jczeng/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:85: RuntimeWarning: invalid value encountered in double_scalars


Phrase label precision: 0.0	recall: 0.000000
Emerging phrase precision: 0.0	recall: 0.000000
Phrase F1 score: nan
[[['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song'], ['spotify_premium'], ['support_chromecast'], ['menu_button'], ['favorite_music']], [['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song'], ['spotify_premium'], ['support_chromecast'], ['menu_button'], ['favorite_music']], [['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song'], ['spotify_premium'], ['support_chromecast'], ['menu_button'], ['favorite_music']], [['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song'], ['spotify_premium'], ['support_chromecast'], ['menu_button'], ['favorite_music']], [['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song'], ['spotify_premium'], ['support_chromecast'], ['menu_button'], ['favorite_music']], [['chromecast_support'], [u'stop_work'], ['music_offline'], [u'suggest_song

In [8]:
import re
if re.search(r'^[-+]?[0-9]*\.?[0-9]+$', '1.44402872553'):
    print "yes"

yes


In [10]:
map(lemmatize, ["2",'1'])

['2', '1']